In [1]:
import csv
import requests
import os
import numpy as np
import pandas as pd 
from urllib.request import urlopen
from bs4 import BeautifulSoup
import geocoder 
from sklearn.cluster import KMeans
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

### Data Obtaining Part 1: Defining States and Capitals

Specifically choose 3 states from each region of the USA. Priority on BBQ states and population count 
Division 1: Massachusetts, Connecticut, Rhode Island
Division 2: New Jersey, New York, Pennsylvania 
Division 3: Illinois, Ohio, Indiana
Division 4: Kansas, Missouri, Nebraska
Division 5: North Carolina, South Carolina, Florida
Division 6: Alabama, Tennessee, Kentucky
Division 7: Texas, Louisiana (50), Oklahoma (27) 
Division 8: Nevada (28), Arizona (5), Colorado (19)
Division 9: California, Hawaii, Washington

https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States
https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population
https://en.wikipedia.org/wiki/Barbecue_in_the_United_States#Kansas_City
https://en.wikipedia.org/wiki/File:Census_Regions_and_Division_of_the_United_States.svg

In [132]:
state_capital = pd.read_csv("us-state-capitals.csv")
state_capital.rename(columns={"description":"capital"}, inplace=True)
state_capital.head()

,name,capital,latitude,longitude
0,Alabama,Montgomery,32.377716,-86.300568
1,Alaska,Juneau,58.301598,-134.420212
2,Arizona,Phoenix,33.448143,-112.096962
3,Arkansas,Little Rock,34.746613,-92.288986
4,California,Sacramento,38.576668,-121.493629


In [142]:
### Create State Capital Panda DF. Contains State/Capital/Division 
state_capital_df = pd.DataFrame(columns=['State', 'Capital', 'Division'] )

### List of states per division 
div_1 = ["Massachusetts", "Connecticut", "Rhode Island"]
div_2 = ["New Jersey", "New York", "Pennsylvania"]
div_3 = ["Illinois", "Ohio", "Indiana"]
div_4 = ["Kansas", "Missouri", "Nebraska"]
div_5 = ["North Carolina", "South Carolina", "Florida"]
div_6 = ["Alabama", "Tennessee", "Kentucky"]
div_7 = ["Texas", "Louisiana", "Oklahoma"]
div_8 = ["Nevada", "Arizona", "Colorado"]
div_9 = ["California", "Hawaii", "Washington"]

combined_div = [ div_1, div_2, div_3, div_4, div_5, div_6, div_7, div_8, div_9]

### Creating each row 
for div in enumerate(combined_div) : 
    for state in div[1] : 
        result = state_capital.loc[ state_capital["name"] == state ]
        
        if result.empty : 
            print("Failed")
            break 
        
        state_capital_df = state_capital_df.append({'State': state,
                                                    'Capital': result.capital.values[0],
                                                    'Division': div[0] + 1}, ignore_index=True)
    if result.empty : 
        break 

print(state_capital_df.shape)
state_capital_df.head()

(27, 3)


,State,Capital,Division
0,Massachusetts,Boston,1
1,Connecticut,Hartford,1
2,Rhode Island,Providence,1
3,New Jersey,Trenton,2
4,New York,Albany,2


### Data Obtaining Part 2: Create GET Request on Foursquare to Obtain Restaurants

In [160]:
### Create Main Restaurant Panda DF 
### Table Entries: ID, Name, Lat/Long, City/State, Category Name, Rating, Number of Rating
### Menu, Pricing,  

restaurant_df = pd.DataFrame(columns=['ID', 'Name', 'Latitude', "Longitude", "State", "City", "Division", "Category Name", "Rating", "Number of Rating"] )

In [26]:
### Foursquare Client ID/Secret and Version
CLIENT_ID = 'TTBNZVFHMNPROOW2YUFMMDCLG1RWVHJURIQEJG5JH2ZG0510' 
CLIENT_SECRET = '4FVF4FFBMZUPRFXCAHLUVYFQFNDOVLR3HNUXIGZX1ECR3KVA'
VERSION = '20191230'

### Category ID matches venues having BBQ Restaurant as their category. 
categoryId= "4bf58dd8d48988d1df931735"
LIMIT = "300"
radius = 100000 

rest_info = []

def restaurant_retrieve (states, cities, divisions, restaurant_df ) : 
    
    for state, city, div in zip(states, cities, divisions) : 

        ### Create URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&categoryId={}&near={},{}&limit={}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            categoryId, 
            city, 
            state,  
            LIMIT, 
            radius)

        ### Make GET Request
        results = requests.get(url).json()
        
        ### Append to DF
        for v in results["response"]["venues"]:
            
            if 
            rest_info.append([v["id"], v["name"], v["location"]["lat"], v["location"]["lng"], 
                  v["location"]["state"], v["location"]["city"], 1, v["categories"][0]["name"], 
                  0, 0])
            
            
        restaurant_df = restaurant_df.append({'ID': state,
                                              'Capital': result.capital.values[0],
                                              'Latitude'
                                              'Division': div[0] + 1}, ignore_index=True)
        



Table Entries
ID, Name, Lat/Long, City/State, Category Name, Menu, Pricing, Rating, Number of Rating 

**Menu and Pricing will probably need to be in a separate table as well 
**Rating will show how well the restaurant is performing 
**Number of rating will show how many people actively rated this restaurant

In [181]:
rest_info = [] 

for v in results["response"]["venues"]: 
    rest_info.append([v["id"], v["name"], v["location"]["lat"], v["location"]["lng"], 
                      v["location"]["state"], v["location"]["city"], 1, v["categories"][0]["name"], 
                      0, 0])



In [188]:
restaurant_df = pd.DataFrame(rest_info)
restaurant_df.columns = ['ID', 'Name', 'Latitude', "Longitude", 
                         'State', "City", "Division", "Category Name", 
                         "Rating", "Number of Rating"]

In [189]:
restaurant_df

,ID,Name,Latitude,Longitude,State,City,Division,Category Name,Rating,Number of Rating
0,518480e4498e95d7e48e35e8,Chuck's Southern Comforts Cafe,41.741519,-87.972836,IL,Darien,1,BBQ Joint,0,0
1,49cfdb98f964a520d45a1fe3,Twin Anchors Restaurant & Tavern,41.912742,-87.638619,IL,Chicago,1,BBQ Joint,0,0
2,52f18c49498e6b5f0500b610,Green Street Smoked Meats,41.883747,-87.648819,IL,Chicago,1,BBQ Joint,0,0
3,40b28c80f964a5204bff1ee3,Sheffield's Beer & Wine Garden,41.941668,-87.654228,IL,Chicago,1,Bar,0,0
4,42d6fc80f964a52027261fe3,The Fireplace Inn,41.909147,-87.634686,IL,Chicago,1,BBQ Joint,0,0
5,50c92495e4b030cbb202e88a,Bub City,41.890195,-87.631050,IL,Chicago,1,BBQ Joint,0,0
6,5b72d3777918710039b278d0,City Barbeque,41.809119,-88.012544,IL,Downers Grove,1,BBQ Joint,0,0
7,5bb7e58f088158002cca36b0,The Red Barn Restaurant & Brewery,42.080638,-87.932739,IL,Mount Prospect,1,American Restaurant,0,0
8,4b36df78f964a520aa3d25e3,Niu Japanese Fusion Lounge,41.891016,-87.618674,IL,Chicago,1,Sushi Restaurant,0,0
9,4acf5b3af964a52034d320e3,The Patio,41.725580,-87.798957,IL,Bridgeview,1,BBQ Joint,0,0


In [176]:
for v in results["response"]["venues"]: 
    # ['id', 'name', 'location', 'categories', 'delivery', 'venuePage', 'referralId', 'hasPerk']
    print(v["categories"][0]["name"])

BBQ Joint
BBQ Joint
BBQ Joint
Bar
BBQ Joint
BBQ Joint
BBQ Joint
American Restaurant
Sushi Restaurant
BBQ Joint
BBQ Joint
BBQ Joint
Steakhouse
Japanese Restaurant
Greek Restaurant
BBQ Joint
BBQ Joint
BBQ Joint
Asian Restaurant
BBQ Joint
Caribbean Restaurant
BBQ Joint
BBQ Joint
BBQ Joint
American Restaurant
Korean Restaurant
Chinese Restaurant
BBQ Joint
BBQ Joint
BBQ Joint
BBQ Joint
BBQ Joint
BBQ Joint
BBQ Joint
Café
BBQ Joint
BBQ Joint
BBQ Joint
BBQ Joint
Japanese Restaurant
BBQ Joint
BBQ Joint
Burger Joint
American Restaurant
BBQ Joint
BBQ Joint
BBQ Joint
Brewery
BBQ Joint
BBQ Joint


In [29]:
print( len(results["response"]["venues"]))

50
